# Data Preperation Program

We are going to read files from different sources and prepare the files for the actual script (The Battle of Neighborhoods Week 2.ipnyb). This is because, some of the data sources have daily quota limits , have network latencies , or need additional pre-processing to get the data ready for the actual algorthim

#### Load all necessary libraries

In [1]:
import io
import requests
import pandas as pd
import numpy as np
from requests import request
import random # library for random number generation
import json
from pandas.io.json import json_normalize

#### libraries for displaying images

In [2]:
from IPython.display import Image 
from IPython.core.display import HTML 
!conda install pip -- yes

Solving environment: failed

PackagesNotFoundError: The following packages are not available from current channels:

  - yes

Current channels:

  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/free/linux-64
  - https://repo.anaconda.com/pkgs/free/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/pro/linux-64
  - https://repo.anaconda.com/pkgs/pro/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [3]:
!pip install geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="battle of neighbourhoods")

     |████████████████████████████████| 102kB 2.9MB/s ta 0:00:011
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/99/45/d1/14954797e2a976083182c2e7da9b4e924509e59b6e5c661061
Successfully built geographiclib


In [4]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total: 

#### download countries geojson file

In [ ]:
!wget --quiet https://data.cityofnewyork.us/Public-Safety/Police-Precincts/78dh-3ptz -O world_countries.json
    
print('GeoJSON file downloaded!')

world_geo = r'world_countries.json' # geojson file

# create a plain world map
world_map = folium.Map(location=[LATITUDE,LONGITUDE], zoom_start=2, tiles='Mapbox Bright')

## Prepare New York Borough & Neighbourhood data file
Input: https://cocl.us/new_york_dataset

In [5]:
def newyork_data():
    url='https://cocl.us/new_york_dataset'
    resp=requests.get(url).json()
    # all data is present in features label
    features=resp['features']
    
    # define the dataframe columns
    column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
    # instantiate the dataframe
    new_york_df = pd.DataFrame(columns=column_names)
    
    for data in features:
        borough = data['properties']['borough'] 
        neighborhood_name = data['properties']['name']
        neighborhood_lattitude = data['geometry']['coordinates'][1]
        neighborhood_longitude = data['geometry']['coordinates'][0]
        new_york_df = new_york_df.append({'Borough': borough, 'Neighborhood': neighborhood_name, 'Latitude': neighborhood_lattitude, 'Longitude': neighborhood_longitude}, ignore_index=True)
    
    return new_york_df 

In [64]:
# get new york data
new_york_df=newyork_data()
print (new_york_df.head())

  Borough Neighborhood   Latitude  Longitude
0   Bronx    Wakefield  40.894705 -73.847201
1   Bronx   Co-op City  40.874294 -73.829939
2   Bronx  Eastchester  40.887556 -73.827806
3   Bronx    Fieldston  40.895437 -73.905643
4   Bronx    Riverdale  40.890834 -73.912585


In [65]:
print (new_york_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 4 columns):
Borough         306 non-null object
Neighborhood    306 non-null object
Latitude        306 non-null float64
Longitude       306 non-null float64
dtypes: float64(2), object(2)
memory usage: 9.7+ KB
None


In [66]:
export_csv = new_york_df.to_csv (r'newyork_borough_data.csv', index = None, header=True) 

## Prepare New York Crime Data file
Input: https://data.cityofnewyork.us/resource/5uac-w243.json

In [76]:
crime_data=[]
crime_data = pd.read_json('https://data.cityofnewyork.us/resource/5uac-w243.json') 
crime_data.head(5)

,cmplnt_num,addr_pct_cd,boro_nm,cmplnt_fr_dt,cmplnt_fr_tm,cmplnt_to_dt,cmplnt_to_tm,crm_atpt_cptd_cd,jurisdiction_code,juris_desc,...,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih,loc_of_occur_desc,hadevelopt,housing_psa,station_name,transit_district,parks_nm
0,857927015,10,MANHATTAN,2019-01-29T00:00:00.000,16:37:00,2019-01-29T00:00:00.000,16:45:00,COMPLETED,0.0,N.Y. POLICE DEPT,...,12,4,10,6,NaN,NaN,NaN,NaN,NaN,NaN
1,479254687,101,QUEENS,2019-03-29T00:00:00.000,17:00:00,2019-03-29T00:00:00.000,17:10:00,COMPLETED,0.0,N.Y. POLICE DEPT,...,51,3,47,59,INSIDE,NaN,NaN,NaN,NaN,NaN
2,320007604,41,BRONX,2019-02-06T00:00:00.000,02:00:00,NaN,NaN,COMPLETED,0.0,N.Y. POLICE DEPT,...,8,5,43,24,INSIDE,NaN,NaN,NaN,NaN,NaN
3,746022144,68,BROOKLYN,2019-01-08T00:00:00.000,22:49:00,2019-01-08T00:00:00.000,22:52:00,COMPLETED,0.0,N.Y. POLICE DEPT,...,10,2,7,41,NaN,NaN,NaN,NaN,NaN,NaN
4,593941718,45,BRONX,2019-03-17T00:00:00.000,05:00:00,2019-03-17T00:00:00.000,05:20:00,COMPLETED,0.0,N.Y. POLICE DEPT,...,43,5,12,28,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
crime_data.info(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 40 columns):
cmplnt_num                     1000 non-null int64
addr_pct_cd                    1000 non-null int64
boro_nm                        995 non-null object
cmplnt_fr_dt                   1000 non-null object
cmplnt_fr_tm                   1000 non-null object
cmplnt_to_dt                   703 non-null object
cmplnt_to_tm                   704 non-null object
crm_atpt_cptd_cd               1000 non-null object
jurisdiction_code              995 non-null float64
juris_desc                     1000 non-null object
ky_cd                          1000 non-null int64
law_cat_cd                     1000 non-null object
ofns_desc                      999 non-null object
patrol_boro                    995 non-null object
pd_cd                          995 non-null float64
pd_desc                        995 non-null object
prem_typ_desc                  990 non-null object
rpt_dt              

In [78]:
crime_data.head(5)
crime_data1 =[]
crime_data1 = crime_data[['addr_pct_cd','boro_nm','pd_desc','latitude','longitude']]

In [79]:
crime_data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
addr_pct_cd    1000 non-null int64
boro_nm        995 non-null object
pd_desc        995 non-null object
latitude       1000 non-null float64
longitude      1000 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 39.2+ KB


In [80]:
newyork_crimes_df= crime_data1.dropna(subset=['boro_nm'])

In [81]:
newyork_crimes_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 995 entries, 0 to 999
Data columns (total 5 columns):
addr_pct_cd    995 non-null int64
boro_nm        995 non-null object
pd_desc        995 non-null object
latitude       995 non-null float64
longitude      995 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 46.6+ KB


In [82]:
newyork_crimes_df.columns = ['Precincts','Borough','Crime','Latitude','Longitude']
newyork_crimes_df['Borough'] = newyork_crimes_df.Borough.astype(str).str.title()
newyork_crimes_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 995 entries, 0 to 999
Data columns (total 5 columns):
Precincts    995 non-null int64
Borough      995 non-null object
Crime        995 non-null object
Latitude     995 non-null float64
Longitude    995 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 46.6+ KB


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [83]:
newyork_crimes_df.head()

,Precincts,Borough,Crime,Latitude,Longitude
0,10,Manhattan,"ASSAULT 2,1,UNCLASSIFIED",40.747777,-74.000398
1,101,Queens,"BURGLARY,RESIDENCE,DAY",40.598538,-73.748560
2,41,Bronx,"ROBBERY,BEGIN AS SHOPLIFTING",40.819886,-73.901227
3,68,Brooklyn,"CONTROLLED SUBSTANCE,INTENT TO",40.624663,-73.999321
4,45,Bronx,ASSAULT 3,40.838545,-73.844583


In [85]:
export_csv = newyork_crimes_df.to_csv (r'newyork_crime_data.csv', index = None, header=True) 

## Prepare Hotels list based on selected Neighbourhood Data file
Input: selected_neighbourhoods_data. This file was prepared by processing Top 50 safe ranked neighbourhoods & Precincts that have crime count less than the mean of total crime count.

In [120]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
selected_neighbourhoods_df = pd.read_csv('selected_neighbourhoods_data.csv', delimiter=',', names = column_names,header=0 )
selected_neighbourhoods_df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Co-op City,40.874294,-73.829939
1,Bronx,Fieldston,40.895437,-73.905643
2,Bronx,Riverdale,40.890834,-73.912585
3,Bronx,Kingsbridge,40.881687,-73.902818
4,Manhattan,Marble Hill,40.876551,-73.910660


In [121]:
selected_neighbourhoods_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 4 columns):
Borough         127 non-null object
Neighborhood    127 non-null object
Latitude        127 non-null float64
Longitude       127 non-null float64
dtypes: float64(2), object(2)
memory usage: 4.1+ KB


In [122]:
# The code was removed by Watson Studio for sharing.

In [136]:
def get_hotels_by_borough(borough, neighbourhood,lat,lng):
    api = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'
    url = api.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, RADIUS, LIMIT)

    results = requests.get(url).json()
    hotel_list_df=[]
    hotel_data=results["response"]['groups'][0]['items']
    try:
        for each_row in hotel_data:
            try:
                hotel_id=each_row['venue']['id']
                hotel_name=each_row['venue']['name']
                try:
                    hotel_category_nm=each_row['venue']['categories'][0]['name']
                    hotel_category_id=each_row['venue']['categories'][0]['id']
                except:
                    hotel_category='Category Not Available'
                hotel_lat = each_row['venue']['location']['lat']
                hotel_lng = each_row['venue']['location']['lng']
                hotel_list_df.append([borough, neighbourhood,lat,lng,hotel_id,hotel_name,hotel_category_nm,hotel_category_id,hotel_lat,hotel_lng])
            except KeyError:
                pass  

        column_names=['Borough','Neighbourhood','NeighbourhoodLat','NeighbourhoodLng','ID','Name','Category_NM','Category_ID','HotelLat','HotelLng']
        hotel_list_df = pd.DataFrame(hotel_list_df,columns=column_names)
    #    print (hotel_list_df.head())
        return hotel_list_df
    except KeyError:
        pass

In [137]:
#column_names=['Borough', 'Neighborhood', 'ID','Name','Latitude','Longtude']
column_names=['Borough','Neighbourhood','Hotel_ID','Hotel_Name','Category_NM','Category_ID','Latitude','Longtude']
hotels_ny=pd.DataFrame(columns=column_names)
count=1

# ny_list_df =selected_neighbourhoods_df[selected_neighbourhoods_df['Neighborhood']=='Co-op City'] 
# ny_list_df =new_york_df[new_york_df['Borough']=='Brooklyn']
ny_list_df =selected_neighbourhoods_df
ny_list = ny_list_df.values.tolist()

for row in ny_list:
    Borough, Neighborhood, Latitude, Longitude=row
    venues = get_hotels_by_borough(Borough,Neighborhood,Latitude,Longitude)
#    found_hotels_df=venues[venues['Category']=='Hotel']   
    found_hotels_df=venues[venues['Category_ID'].isin(['4bf58dd8d48988d1fa931735', '4bf58dd8d48988d1f8931735', '5bae9231bedf3950379f89cb', '4bf58dd8d48988d1fb931735', '4bf58dd8d48988d12f951735'])]
    print('Processed '+Borough+ ' / '+Neighborhood+ ', Hotel count: '+ str(len(found_hotels_df)))
    for each_row in found_hotels_df.values.tolist():
#        print (borough+ ' / '+neighborhood+ ' / '+ name)
        borough,neighborhood,nlat,nlng,hid, name , cat_nm, cat_id,hlat,hlng =each_row
        hotels_ny = hotels_ny.append({'Borough': borough,
                                                'Neighbourhood': neighborhood, 
                                                'Hotel_ID': hid,
                                                'Hotel_Name' : name,
                                                'Category_NM' :cat_nm,
                                                'Category_ID' : cat_id,
                                                'Latitude': hlat,
                                                'Longtude' : hlng
                                               }, ignore_index=True)
    count+=1


Processed Bronx / Co-op City, Hotel count: 1
Processed Bronx / Fieldston, Hotel count: 0
Processed Bronx / Riverdale, Hotel count: 0
Processed Bronx / Kingsbridge, Hotel count: 0
Processed Manhattan / Marble Hill, Hotel count: 0
Processed Bronx / City Island, Hotel count: 1
Processed Bronx / Throgs Neck, Hotel count: 1
Processed Bronx / Spuyten Duyvil, Hotel count: 0
Processed Bronx / Pelham Bay, Hotel count: 1
Processed Brooklyn / Bay Ridge, Hotel count: 0
Processed Brooklyn / Sunset Park, Hotel count: 0
Processed Brooklyn / Greenpoint, Hotel count: 2
Processed Brooklyn / Brighton Beach, Hotel count: 0
Processed Brooklyn / Sheepshead Bay, Hotel count: 0
Processed Brooklyn / Crown Heights, Hotel count: 0
Processed Brooklyn / Kensington, Hotel count: 0
Processed Brooklyn / Windsor Terrace, Hotel count: 0
Processed Brooklyn / Prospect Heights, Hotel count: 0
Processed Brooklyn / Williamsburg, Hotel count: 2
Processed Brooklyn / Cobble Hill, Hotel count: 2
Processed Brooklyn / Carroll Gar

In [138]:
print (hotels_ny.head())

    Borough Neighbourhood                  Hotel_ID  \
0     Bronx    Co-op City  54932887498ee0902b1ed511   
1     Bronx   City Island  54932887498ee0902b1ed511   
2     Bronx   Throgs Neck  54932887498ee0902b1ed511   
3     Bronx    Pelham Bay  54932887498ee0902b1ed511   
4  Brooklyn    Greenpoint  4ab04eb9f964a520fe6620e3   

                                          Hotel_Name Category_NM  \
0  Residence Inn by Marriott New York The Bronx a...       Hotel   
1  Residence Inn by Marriott New York The Bronx a...       Hotel   
2  Residence Inn by Marriott New York The Bronx a...       Hotel   
3  Residence Inn by Marriott New York The Bronx a...       Hotel   
4                                   The Bowery Hotel       Hotel   

                Category_ID   Latitude   Longtude  
0  4bf58dd8d48988d1fa931735  40.850020 -73.842579  
1  4bf58dd8d48988d1fa931735  40.850020 -73.842579  
2  4bf58dd8d48988d1fa931735  40.850020 -73.842579  
3  4bf58dd8d48988d1fa931735  40.850020 -73.842579  


In [139]:
# hotels_ny = hotels_ny.drop_duplicates(subset='ID', keep='first')
hotels_ny.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 8 columns):
Borough          41 non-null object
Neighbourhood    41 non-null object
Hotel_ID         41 non-null object
Hotel_Name       41 non-null object
Category_NM      41 non-null object
Category_ID      41 non-null object
Latitude         41 non-null float64
Longtude         41 non-null float64
dtypes: float64(2), object(6)
memory usage: 2.7+ KB


In [140]:
export_csv = hotels_ny.to_csv (r'newyork_hotels_data.csv', index = None, header=True) 